# Lecture 1-2. 구현 실습  

예시 문제 링크 
- [1번: 백준 2578번](https://www.acmicpc.net/problem/2578)
- [2번: 백준 12100번](https://www.acmicpc.net/problem/12100)
- [3번: 코드트리 마법의 숲 탐색](https://www.codetree.ai/ko/frequent-problems/samsung-sw/problems/magical-forest-exploration/description)
- [4번: 코드트리 청소는 즐거워](https://www.codetree.ai/ko/frequent-problems/samsung-sw/problems/cleaning-is-joyful/description)
- [5번: 코드트리 미생물 연구](https://www.codetree.ai/ko/frequent-problems/samsung-sw/problems/microbial-research/description)
- [6번: 코드트리 민트초코 우유](https://www.codetree.ai/ko/frequent-problems/samsung-sw/problems/mint-choco-milk/description)
- [7번: 코드트리 메두사와 전사들](https://www.codetree.ai/ko/frequent-problems/samsung-sw/problems/medusa-and-warriors/description)


## 1번 정답

- 풀이 방법: 
  - 숫자 → 좌표 매핑을 만들어 둔 뒤, 사회자가 번호를 부를 때마다 해당 칸을 체크하고 행/열/대각선의 완성 여부를 빠르게 세는 것입니다. 3줄 이상 완성되는 순간의 호출 횟수를 출력하면 끝!
  - O(25)로 깔끔하게 풀립니다

In [ ]:
import sys

# ----- 입력 -----
board = [list(map(int, sys.stdin.readline().split())) for _ in range(5)]
calls = []
for _ in range(5):
    calls += list(map(int, sys.stdin.readline().split()))  # 총 25개

# 숫자 -> (r, c) 좌표 매핑
pos = {}
for r in range(5):
    for c in range(5):
        pos[board[r][c]] = (r, c)

# 행/열/대각 카운터와 완료 체크
row = [0]*5
col = [0]*5
diag = [0, 0]  # [메인 대각(r==c), 반대각(r+c==4)]
line_done = 0

# 같은 줄을 중복 카운트하지 않게 완료 여부 기록
row_done = [False]*5
col_done = [False]*5
diag_done = [False, False]

for i, num in enumerate(calls, start=1):  # i = 호출 횟수(1~25)
    r, c = pos[num]

    row[r] += 1
    col[c] += 1
    if r == c:
        diag[0] += 1
    if r + c == 4:
        diag[1] += 1

    # 이번 호출로 막 완성된 줄만 카운트 추가
    if row[r] == 5 and not row_done[r]:
        line_done += 1
        row_done[r] = True
    if col[c] == 5 and not col_done[c]:
        line_done += 1
        col_done[c] = True
    if r == c and diag[0] == 5 and not diag_done[0]:
        line_done += 1
        diag_done[0] = True
    if r + c == 4 and diag[1] == 5 and not diag_done[1]:
        line_done += 1
        diag_done[1] = True

    if line_done >= 3:
        print(i)
        break


## 2번 정답 

- 풀이 방법 
  - 풀이 핵심은 완전탐색(DFS, 깊이 ≤ 5) + 정확한 한 번 병합 규칙입니다.
  - 4가지 방향(←↑→↓)으로 최대 5번: 경우의 수는 $4^{5} = 1024$ -> 충분히 전부 탐색 가능. 
  - 각 이동에서 한 칸당 병합은 1번만 일어나도록 주의(연속 2개만 합치고, 합쳐진 타일은 그 이동에서 다시 합쳐지지 않음).
  - 이동으로 보드가 변하지 않으면 가지치기.

In [ ]:
import sys
input = sys.stdin.readline

N = int(input().strip())
board0 = [list(map(int, input().split())) for _ in range(N)]

def merge_line_left(line):
    """한 줄을 왼쪽으로 당기며 2048 병합 규칙 적용"""
    nonzero = [x for x in line if x != 0]
    merged = []
    i = 0
    while i < len(nonzero):
        if i + 1 < len(nonzero) and nonzero[i] == nonzero[i + 1]:
            merged.append(nonzero[i] * 2)
            i += 2
        else:
            merged.append(nonzero[i])
            i += 1
    # 뒤를 0으로 채워 길이 N 유지
    merged += [0] * (N - len(merged))
    return merged

def move(board, d):
    """
    d: 0=LEFT, 1=UP, 2=RIGHT, 3=DOWN
    보드를 해당 방향으로 한 번 움직인 새 보드 반환
    """
    newb = [row[:] for row in board]

    if d == 0:  # LEFT
        for r in range(N):
            newb[r] = merge_line_left(newb[r])

    elif d == 2:  # RIGHT
        for r in range(N):
            rev = list(reversed(newb[r]))
            rev = merge_line_left(rev)
            newb[r] = list(reversed(rev))

    elif d == 1:  # UP
        for c in range(N):
            col = [newb[r][c] for r in range(N)]
            col = merge_line_left(col)
            for r in range(N):
                newb[r][c] = col[r]

    else:  # d == 3, DOWN
        for c in range(N):
            col = [newb[r][c] for r in range(N)][::-1]
            col = merge_line_left(col)
            col = col[::-1]
            for r in range(N):
                newb[r][c] = col[r]

    return newb

def max_tile(board):
    return max(max(row) for row in board)

ans = max_tile(board0)

def dfs(board, depth):
    global ans
    ans = max(ans, max_tile(board))
    if depth == 5:
        return
    for d in range(4):
        nb = move(board, d)
        if nb != board:        # 변화 없으면 가지치기
            dfs(nb, depth + 1)

dfs(board0, 0)
print(ans)


## 3번 정답

- 풀이
  - 시뮬레이션 + 연결 전파가 핵심!
  - 숲 크기 R×C, 골렘 K개.
  - 골렘은 십자(+) 모양: 중심 + 상하좌우 1칸(총 5칸). 출구는 중심에서 바라보는 방향의 이웃 한 칸.
  - 위에서 떨어지며 아래 → 왼쪽-아래(출구 좌회전) → 오른쪽-아래(출구 우회전) 순으로 시도. 셋 다 불가면 정지.
  - 보드는 시뮬을 쉽게 하려고 윗쪽에 3행을 더 붙인 격자(R+3 × C) 를 쓴다. 최종적으로 십자 4팔 중 한 칸이라도 원래 숲(행 인덱스 3~R+2) 밖이면 전체 숲 초기화(전부 비움) 후 해당 골렘은 무시.
  - 각 골렘이 자리잡으면, 그 골렘을 포함해 출구로 접속 가능한 연결 컴포넌트 안에서 도달 가능한 **최대 행(원래 숲 기준, 0~R-1)**을 구해 합산.

In [ ]:
import sys
from collections import deque

input = sys.stdin.readline

# 방향: 0=북,1=동,2=남,3=서  (출구가 가리키는 방향)
DX = [-1, 0, 1, 0]
DY = [0, 1, 0, -1]

# 아래 직하/좌하/우하 이동 시, 비어있어야 하는 상대 좌표 집합
# (중심 기준 상대 좌표들이 모두 비어있어야 해당 이동 가능)
# 아래로: 중심 아래 2칸, 좌/우 아래 1칸(십자 팔이 내려가며 걸리지 않도록)
TX = [2, 1, 1]; TY = [0, -1, 1]
# 좌하: 왼쪽으로 틀며 내려갈 때 필요한 빈칸(모서리 간섭 방지)
LX = [0, 1, -1, 1, 2]; LY = [-2, -1, -1, -2, -1]
# 우하
RX = [0, 1, -1, 1, 2]; RY = [2, 1, 1, 2, 1]

# 출구 접속을 전파할 때, 한 골렘 중심 주변에서 "출구 셀"을 탐색할 8방 이웃
EX = [-2, -1, 0, 1, 2, 1, 0, -1]
EY = [0, 1, 2, 1, 0, -1, -2, -1]

R, C, K = map(int, input().split())

# 시뮬은 R 위에 3행 더해서 (R+3)×C 보드 사용
H = R + 3

# arr: 골렘 몸(십자 5칸)의 골렘 ID 기록(0=빈칸)
# exit_map: 출구 칸에 골렘 ID 기록(0=없음)
arr = [[0]*C for _ in range(H)]
exit_map = [[0]*C for _ in range(H)]

# 각 골렘 정보와 결과 합
units = [None]*(K+1)  # (x, y, dir) 저장용
max_row = [0]*(K+1)   # 각 골렘 컴포넌트의 "도달 가능한 최대 행"(원래 숲 기준 0~R-1)
answer = 0

def in_range(x, y):
    return 0 <= x < H and 0 <= y < C

def in_forest(x, y):
    # 원래 숲 범위: 행 3..H-1(=R+2)
    return 3 <= x < H and 0 <= y < C

def can_move(x, y, xofs, yofs):
    for dx, dy in zip(xofs, yofs):
        nx, ny = x + dx, y + dy
        if not in_range(nx, ny) or arr[nx][ny] != 0:
            return False
    return True

def drop_and_settle(x, y, d):
    # (x,y,d)에서 아래/좌하/우하를 우선순위대로 반복 시도
    while True:
        if can_move(x, y, TX, TY):
            x += 1
            continue
        if can_move(x, y, LX, LY):  # 좌하: 출구 좌회전
            d = (d + 3) % 4
            x += 1; y -= 1
            continue
        if can_move(x, y, RX, RY):  # 우하: 출구 우회전
            d = (d + 1) % 4
            x += 1; y += 1
            continue
        break
    return x, y, d

def place_or_reset(id_, x, y, d):
    # 십자 네 팔이 원래 숲 범위를 벗어나면 전체 초기화
    for dir4 in range(4):
        ax, ay = x + DX[dir4], y + DY[dir4]
        if not in_forest(ax, ay):
            # 전체 리셋
            for i in range(H):
                for j in range(C):
                    arr[i][j] = 0
                    exit_map[i][j] = 0
            return False

    # 출구 표시
    ex, ey = x + DX[d], y + DY[d]
    exit_map[ex][ey] = id_

    # 몸통(십자) 표시
    arr[x][y] = id_
    for dir4 in range(4):
        ax, ay = x + DX[dir4], y + DY[dir4]
        arr[ax][ay] = id_

    units[id_] = (x, y, d)
    return True

def compute_component_maxrow(start_id):
    """start_id 골렘을 기준으로 출구-접속 가능한 골렘들에 대해
       도달 가능한 최대 행(원래 숲 좌표계) 값을 전파(BFS)하며 갱신"""
    # 기본: 중심의 '숲 내' 행 인덱스 = x-3
    sx, sy, sd = units[start_id]
    base = sx - 1  # (x-1) == (x-3 + 2)와 동일하게 보정되며, 표준 풀이에서 쓰는 기준
    # 직관적으론: 중심 기준으로 도달 가능한 최대 행을 저장(문항 정의에 맞춰 합산)

    # 먼저, 현재 골렘의 출구 주변 4방에 인접한 '몸'이 있으면 그 골렘으로부터 더 큰 최대값을 물려받음
    best = sx - 1
    ex, ey = sx + DX[sd], sy + DY[sd]
    for t in range(4):
        nx, ny = ex + DX[t], ey + DY[t]
        if in_range(nx, ny) and arr[nx][ny] != 0 and arr[nx][ny] != start_id:
            nid = arr[nx][ny]
            best = max(best, max_row[nid])

    # 시작 골렘의 현재 best 기록
    if best > max_row[start_id]:
        max_row[start_id] = best

    # 이제 출구가 "맞닿아 있는" 골렘들로 최대 행을 전파 (출구 맵 기준 8방)
    q = deque([start_id])
    seen = {start_id}
    while q:
        gid = q.popleft()
        gx, gy, gd = units[gid]
        # 중심 8방의 출구 칸들을 보며, 출구가 있는 다른 골렘을 찾는다
        for k in range(8):
            nx, ny = gx + EX[k], gy + EY[k]
            if in_range(nx, ny) and exit_map[nx][ny] != 0:
                nid = exit_map[nx][ny]
                if nid not in seen:
                    seen.add(nid)
                    # 더 큰 최대행을 전파
                    if max_row[gid] > max_row[nid]:
                        max_row[nid] = max_row[gid]
                    q.append(nid)

def solve():
    global answer
    for gid in range(1, K+1):
        c, d = map(int, input().split())
        # 시작 지점: (1, c-1)에서 낙하 시작 (보드 상단 여유행 고려)
        x, y, dir_ = 1, c-1, d
        x, y, dir_ = drop_and_settle(x, y, dir_)

        # 놓을 수 없으면 전체 보드 초기화되고, 이 골렘은 스킵
        if not place_or_reset(gid, x, y, dir_):
            continue

        # 이 골렘 기준으로 최대행 계산 및 출구 접속 전파
        compute_component_maxrow(gid)
        answer += max_row[gid]

    print(answer)

if __name__ == "__main__":
    solve()
